### Script for adding CESM-LENS perturbation to ERA5 2D files (streamlined)
### date created: 13 May 2025
### author: doughert@ucar.edu

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
# import netCDF4 as nc
# from netCDF4 import Dataset, num2date
#from datetime import datetime, date, timedelta
import glob
import xarray as xr

### change the year, month, and days from ERA5 files to look at, as well export path

In [2]:
yr_mo = '201709'
days = ['15', '16', '17','18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29','30',]
export_dir = '/'
input_dir = '/'

#### list variable names from perturbed LENS2 output - DON'T CHANGE BELOW HERE

In [3]:
var_list = ['SST', 'TS',  'PS', 'PSL', 'ICEFRAC', 'SNOWHLND']

#### list ERA5 variables that will be perturbed

In [4]:
era5_sfc = '/glade/campaign/collections/rda/data/ds633.0/e5.oper.an.sfc/'
var_era = ['128_034_sstk.ll025sc', '128_235_skt.ll025sc','128_134_sp.ll025sc', '128_151_msl.ll025sc', '128_031_ci.ll025sc', '128_141_sd.ll025sc']
era5_name = ['SSTK', 'SKT', 'SP', 'MSL', 'CI', 'SD']
param_ids = ["34.128", "235.128", "134.128", "151.128", "31.128", "141.128"]

### definitions- 1st one to replace ERA5 files with perturbed value, second to format the output based on what MPAS expects

In [5]:
def replace_era5_var(orig_era5, var, ptb_era5):
    era5_rp = orig_era5.assign({var:ptb_era5})     
    
    return(era5_rp)

In [6]:
def fmt_files(var, pid, zaxis_file):
    from cdo import Cdo
    cdo = Cdo(cdo='/glade/u/home/doughert/miniconda3/envs/pangeo3/bin/cdo')
    ### change missing value to nan
    substr = yr_mo+'3023'
    add_str = 'nan'

    for file in sorted(glob.glob(export_dir+'era5.sfc.'+var+'.*.nc')):
        stridx = file.index(substr)
        # insert nan after date for file output name
        newfname = str(file[:stridx+10]) + add_str + str(file[stridx+10:])
        print(newfname)
        cdo.setmissval("nan", input=file, output=newfname)

    substr = 'nan'
    add_str2 = 'miss'
    ## change missing value to -999
    for file in sorted(glob.glob(export_dir+'era5.sfc.'+var+'.*nan.nc')):
        stridx = file.index(substr)
        newfname = str(file[:stridx]) + add_str2 + str(file[stridx+3:])
        print(newfname)
        cdo.setmissval("-999.9", input=file, output=newfname)

    ## select first variable
    substr = 'miss'
    add_str3 = 'var1'
    for file in sorted(glob.glob(export_dir+'era5.sfc.'+var+'.*miss.nc')):
        stridx = file.index(substr)
        newfname = str(file[:stridx]) + add_str3 + str(file[stridx+4:]) 
        newfname = newfname[:-2]+'grb'
        #newfname = newfname
        print(newfname)
        #cdo.selparam("-1", input=file, output=newfname,)
        cdo.selparam("-1", input=file, output=newfname, options='-f grb')

    
    ## change parameter id code 
    substr = 'var1'
    add_str3 = 'final'
    for file in sorted(glob.glob(export_dir+'era5.sfc.'+var+'*var1.grb')):
        stridx = file.index(substr)
        newfname = str(file[:stridx]) + add_str3 + str(file[stridx+4:]) 
        print(newfname)
        cdo.setparam(pid, input=file, output=newfname, )
        
    ## change soil axis for soil variables only    
    if var.startswith('STL'):
        substr = 'pid'
        add_str3 = 'zaxis1'
        for file in sorted(glob.glob(export_dir+'era5.sfc.'+var+'*pid.grb')):
            stridx = file.index(substr)
            newfname = str(file[:stridx]) + add_str3 + str(file[stridx+3:]) 
            print(newfname)
            cdo.setzaxis(export_dir+zaxis_file, input=file, output=newfname)
                

### this routine opens delta from LENS2, opens ERA5 for same variable, interpolates ERA5 and CESM to match
### then adds delta to ERA5, and export file with perturbed variables

In [7]:
import pyresample

for i in range(len(var_list)):
    # open deltas
    delta_var = xr.open_dataset(input_dir+'LENS2-Sept_2070-2100_1991-2021_delta_'+var_list[i]+'.nc')[var_list[i]]
    cesm_lat = delta_var.lat
    cesm_lon = delta_var.lon
    
    # open era5 data for same variable
    era5_var_days = xr.open_dataset(era5_sfc+yr_mo+'/e5.oper.an.sfc.'+var_era[i]+'.'+yr_mo+'0100_'+yr_mo+'3023.nc')
    era5_lat = era5_var_days.latitude
    era5_lon = era5_var_days.longitude
    era5_lon2d, era5_lat2d  = np.meshgrid(era5_lon.values, era5_lat.values)
    
    # horizontally reproject data
    cesm_lon2d, cesm_lat2d = np.meshgrid(cesm_lon.values, cesm_lat.values)
    # change CESM lons to be from -180 to 180 instead of 0 to 360
    cesm_lon2d_new = pyresample.utils.check_and_wrap(cesm_lon2d, cesm_lat2d)[0]
    # create presample object of the CESM data
    orig_def = pyresample.geometry.SwathDefinition(cesm_lon2d_new, cesm_lat2d)
    # change ERA-5 lons to be from -180 to 180 instead of 0 to 360
    era5_lon2d_new = pyresample.utils.check_and_wrap(era5_lon2d, era5_lat2d)[0]
    # presample object of ERA-5 data
    targ_def = pyresample.geometry.SwathDefinition(era5_lon2d_new, era5_lat2d)

    ### reproject CESM data to ERA-5 grid (CESM grid ~ 110 km spacing, so change radius of influence to be larger, about 100 km)
    re_cesm_delta_var = pyresample.kd_tree.resample_nearest(orig_def, delta_var.values, targ_def, radius_of_influence=110000, fill_value=np.nan)

    # add delta to ERA5 data
    era5_times = len(era5_var_days[era5_name[i]])
    era5_var_ptb = era5_var_days[era5_name[i]]+([np.nan_to_num(re_cesm_delta_var, nan=0.0)]*era5_times) 

    # replace ERA5 data with perturbed LENS data
    era5_var_change = replace_era5_var(era5_var_days, era5_name[i], era5_var_ptb)

    # export modified ERA5
    era5_var_change.to_netcdf(export_dir+'era5.sfc.'+era5_name[i]+'.'+yr_mo+'0100_'+yr_mo+'3023.nc')

    fmt_files(era5_name[i], param_ids[i], zaxis_file=[])

/glade/derecho/scratch/doughert/ERA5_perturb/201709/2d/era5.sfc.SSTK.2017090100_2017093023nan.nc
/glade/derecho/scratch/doughert/ERA5_perturb/201709/2d/era5.sfc.SSTK.2017090100_2017093023miss.nc
/glade/derecho/scratch/doughert/ERA5_perturb/201709/2d/era5.sfc.SSTK.2017090100_2017093023var1.grb
/glade/derecho/scratch/doughert/ERA5_perturb/201709/2d/era5.sfc.SSTK.2017090100_2017093023pid.grb
/glade/derecho/scratch/doughert/ERA5_perturb/201709/2d/era5.sfc.SKT.2017090100_2017093023nan.nc
/glade/derecho/scratch/doughert/ERA5_perturb/201709/2d/era5.sfc.SKT.2017090100_2017093023miss.nc
/glade/derecho/scratch/doughert/ERA5_perturb/201709/2d/era5.sfc.SKT.2017090100_2017093023var1.grb
/glade/derecho/scratch/doughert/ERA5_perturb/201709/2d/era5.sfc.SKT.2017090100_2017093023pid.grb
/glade/derecho/scratch/doughert/ERA5_perturb/201709/2d/era5.sfc.SP.2017090100_2017093023nan.nc
/glade/derecho/scratch/doughert/ERA5_perturb/201709/2d/era5.sfc.SP.2017090100_2017093023miss.nc
/glade/derecho/scratch/doughe

### repeat for soil temperature fields - this splits up TSOI field based on layers in ERA5 soil

In [8]:
var= 'TSOI'

delta_var = xr.open_dataset(input_dir+'LENS2-Sept_2070-2100_1991-2021_delta_'+var+'.nc')[var]
cesm_lat = delta_var.lat
cesm_lon = delta_var.lon

delta_tsoi1 = delta_var.sel(levgrnd=slice(9.9999998e-03, 9.0000004e-02)) #0-9 cm (0-7 ERA5)
delta_tsoi2 = delta_var.sel(levgrnd=slice(9.0000004e-02, 2.5999999e-01)) #9-26 (7-28 ERA5)
delta_tsoi3 = delta_var.sel(levgrnd=slice(2.5999999e-01, 1.0599999e+00)) #26-100 (28-100 ERA5)
delta_tsoi4 = delta_var.sel(levgrnd=slice( 1.0599999e+00, 2.5000000e+00)) #100-255 (100-255 ERA5)
delta_tsoi1_mean = delta_tsoi1.mean(dim='levgrnd')
delta_tsoi2_mean = delta_tsoi2.mean(dim='levgrnd')
delta_tsoi3_mean = delta_tsoi3.mean(dim='levgrnd')
delta_tsoi4_mean = delta_tsoi4.mean(dim='levgrnd')

In [9]:
var_soil = [delta_tsoi1_mean, delta_tsoi2_mean, delta_tsoi3_mean, delta_tsoi4_mean]

In [10]:
var_era = [ '128_139_stl1.ll025sc', '128_170_stl2.ll025sc', '128_183_stl3.ll025sc', '128_236_stl4.ll025sc',]
era5_name = ['STL1', 'STL2', 'STL3', 'STL4',]
param_ids = ["139.128", "170.128", "183.128", "236.128",]
zaxis_file = ['soil_axis1', 'soil_axis2', 'soil_axis3', 'soil_axis4']

In [11]:
import pyresample

for i in range(len(var_soil)):
    
    # open era5 data for same variable
    era5_var_days = xr.open_dataset(era5_sfc+yr_mo+'/e5.oper.an.sfc.'+var_era[i]+'.'+yr_mo+'0100_'+yr_mo+'3023.nc')
    era5_lat = era5_var_days.latitude
    era5_lon = era5_var_days.longitude
    era5_lon2d, era5_lat2d  = np.meshgrid(era5_lon.values, era5_lat.values)
    
    # horizontally reproject data
    cesm_lon2d, cesm_lat2d = np.meshgrid(cesm_lon.values, cesm_lat.values)
    # change CESM lons to be from -180 to 180 instead of 0 to 360
    cesm_lon2d_new = pyresample.utils.check_and_wrap(cesm_lon2d, cesm_lat2d)[0]
    # create presample object of the CESM data
    orig_def = pyresample.geometry.SwathDefinition(cesm_lon2d_new, cesm_lat2d)
    # change ERA-5 lons to be from -180 to 180 instead of 0 to 360
    era5_lon2d_new = pyresample.utils.check_and_wrap(era5_lon2d, era5_lat2d)[0]
    # presample object of ERA-5 data
    targ_def = pyresample.geometry.SwathDefinition(era5_lon2d_new, era5_lat2d)

    ### reproject CESM data to ERA-5 grid (CESM grid ~ 110 km spacing, so change radius of influence to be larger, about 100 km)
    re_cesm_delta_var = pyresample.kd_tree.resample_nearest(orig_def, var_soil[i].values, targ_def, radius_of_influence=110000, fill_value=np.nan)

    # add delta to ERA5 data
    era5_times = len(era5_var_days[era5_name[i]])
    era5_var_ptb = era5_var_days[era5_name[i]]+([np.nan_to_num(re_cesm_delta_var, nan=0.0)]*era5_times) 

    # replace ERA5 data with perturbed LENS data
    era5_var_change = replace_era5_var(era5_var_days, era5_name[i], era5_var_ptb)

    # export modified array
    era5_var_change.to_netcdf(export_dir+'era5.sfc.'+era5_name[i]+'.'+yr_mo+'0100_'+yr_mo+'3023.nc')

    fmt_files(era5_name[i], param_ids[i], zaxis_file[i])

/glade/derecho/scratch/doughert/ERA5_perturb/201709/2d/era5.sfc.STL1.2017090100_2017093023nan.nc
/glade/derecho/scratch/doughert/ERA5_perturb/201709/2d/era5.sfc.STL1.2017090100_2017093023miss.nc
/glade/derecho/scratch/doughert/ERA5_perturb/201709/2d/era5.sfc.STL1.2017090100_2017093023var1.grb
/glade/derecho/scratch/doughert/ERA5_perturb/201709/2d/era5.sfc.STL1.2017090100_2017093023pid.grb
/glade/derecho/scratch/doughert/ERA5_perturb/201709/2d/era5.sfc.STL1.2017090100_2017093023zaxis1.grb
